# Aprendizaje Automático - Intermedio

## Pipelines

### Introducción

**Pipelines** son una forma simple de mantener organizado el preprocesamiento de datos y el código de modelado. Específicamente, un pipeline agrupa los pasos de preprocesamiento y modelado para poder usar todo el paquete como si fuera un solo paso.

Algunos beneficios importantes de los pipelines son los siguientes:

+ **Código más limpio**: el cálculo de datos en cada paso del preprocesamiento puede ser complicado. Con un pipeline no necesitaremos realizar un seguimiento manual de nuestros datos de entrenamiento y validación en cada paso.
+ **Menos errores**: hay menos oportunidades para aplicar mal un paso u olvidar un paso de preprocesamiento.
+ **Más fácil de industrializar**: puede ser sorprendentemente difícil hacer la transición de un modelo de un prototipo a algo desplegable a escala. No entraremos en las muchas preocupaciones relacionadas aquí, pero los pipelines pueden ayudar.
+ **Más opciones para la validación del modelo**: veremos un ejemplo en el próximo tutorial, que cubre la validación cruzada.

### Ejemplo

En el ejemplo, trabajaremos con el conjunto de datos de [Melbourne Housing](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lee los datos
data = pd.read_csv('./input/melbourne-housing-snapshot/melb_data.csv')

# Separa el objetivo de los predictores
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide los datos en subconjuntos de entrenamiento y validación
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinalidad" significa el número de valores únicos en una columna
# Selecciona las columnas categóricas con relativamente baja cardinalidad (conveniente pero arbitrariamente)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Selecciona las columnas numéricas
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Mantiene solo las columnas seleccionadas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Echamos un vistazo a los datos de entrenamiento con el método head(). Observa que los datos contienen datos categóricos y columnas con valores ausentes. ¡Con un pipeline es fácil lidiar con ambos!

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


Vamos a construir un pipeline en tres pasos.

#### Paso 1: definir los pasos de preprocesamiento

De manera similar a cómo un pipeline agrupa los pasos de preprocesamiento y modelado, utilizamos la clase `ColumnTransformer` para agrupar diferentes pasos de preprocesamiento. El código a continuación:

+ imputa valores ausentes en datos **numéricos**, y
+ imputa los valores faltantes y aplica una codificación one-hot a los datos **categóricos**.

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocesado para datos numéricos
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocesado para datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocesamiento en paquete para datos numéricos y categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

#### Paso 2: Definir el modelo

A continuación, definimos un modelo de random forest con la clase `RandomForestRegressor`.

In [4]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

#### Paso 3: crear y evaluar el pipeline

Finalmente, usamos la clase `Pipeline` para definir un pipeline que agrupa los pasos de preprocesamiento y modelado. Hay algunas cosas importantes a tener en cuenta:

+ Con el pipeline preprocesamos los datos de entrenamiento y ajustamos el modelo en una sola línea de código. (*Por el contrario, sin un pipeline, tenemos que hacer la imputación, la codificación one-hot y el entrenamiento del modelo en pasos separados. ¡Esto se vuelve especialmente complicado si tenemos que lidiar con variables numéricas y categóricas!*)
+ Con el pipeline, suministramos las características no procesadas en `X_valid` al comando `predic()`, y el pipeline preprocesa automáticamente las características antes de generar predicciones. (*Sin embargo, sin un pipeline, debemos recordar preprocesar los datos de validación antes de hacer predicciones*).

In [5]:
from sklearn.metrics import mean_absolute_error

# Preprocesamiento y código de modelado en paquete en un pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocesamiento de datos de entrenamiento, entrenamiento del modelo
my_pipeline.fit(X_train, y_train)

# Preprocesamiento de datos de validación, obtiene predicciones
preds = my_pipeline.predict(X_valid)

# Evaluación del modelo
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


### Conclusión

Los pipelines son valiosos para limpiar el código de aprendizaje automático y evitar errores y son especialmente útiles para flujos de trabajo con preprocesamiento de datos sofisticado.

## Ejercicio

En este ejercicio utilizaremos **pipelines** para mejorar la eficiencia de nuestro código de aprendizaje automático. Trabajaremos con datos de [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Vamos a cargar los conjuntos de entrenamiento y validación en `X_train`, `X_valid`, `y_train`, e `y_valid`.  El conjunto de pruebas es cargado en `X_test`.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leer los datos
X_full = pd.read_csv('./input/train.csv', index_col='Id')
X_test_full = pd.read_csv('./input/test.csv', index_col='Id')

# Elimina las filas con objetivos ausentes, separa el objetivo de los predictores
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Separamos los datos de validación a partir de los datos de entrenamiento
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinalidad" significa el número de valores únicos en una columna
# Selecciona las columnas categóricas con relativamente baja cardinalidad (conveniente pero arbitrariamente)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Selecciona columnas numéricas
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Mantiene solo las columnas seleccionadas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [7]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocesamiento de datos numéricos
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocesamiento de datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocesamiento en paquete de datos numéricos y categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define modelo
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Preprocesamiento y codigo de modelado en paquete en un pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocesamiento de datos de entrenamiento, entrenamiento de modelo
clf.fit(X_train, y_train)

# Preprocesamiento de datos de validación, obtener predicciones
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17861.780102739725


El código produce un valor alrededor de 17862 para el error absoluto medio (MAE). En el siguiente paso, modificaremos el código para hacerlo mejor.

### Paso 1: mejorar el rendimiento

#### Parte A

Defina tus propios pasos de preprocesamiento y el modelo de randiom forest. Completa los valores para las siguientes variables:
- `numerical_transformer`
- `categorical_transformer`
- `modelo`

In [9]:
# Preprocesamiento de datos numéricos
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocesamiento de datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define modelo
model = RandomForestRegressor(n_estimators=100, random_state=0)

#### Parte B

In [10]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 17621.3197260274


### Paso 2: Generar predicciones de prueba

Ahora usaremos nuestro modelo entrenado para generar predicciones con los datos de prueba.

In [11]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test_full)

In [12]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('./output/submission.csv', index=False)